<a href="https://colab.research.google.com/github/halldm2000/NOAA-AI-2020-TUTORIAL/blob/master/04_Classification/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Classify Tropical Cyclones by Category

**Download data from a shared google drive** (~2 min)

In [ ]:
%%time
!gdown https://drive.google.com/uc?id=1-CG3sF1JIi_PGDo6stwwmXiTFJuEUqfU
!echo "Extracting..."
!tar -xzf cyclones.tar.gz

**Ensure NetCDF4 is installed, so we can read the data from a file**

In [ ]:
!pip install netcdf4

**Plot some data to make sure things are working correctly**

In [ ]:
import glob, xarray, numpy as np, matplotlib.pyplot as plt

files = glob.glob("crop_64/CAT2/*.nc");
for file in files[:1]:

  # open netcdf file usiing xarray
  print("file =",file)
  ds   = xarray.open_dataset(file);
  wind = np.sqrt(ds["u"].data**2 + ds["v"].data**2) # compute wind speed from u,v

  # plot each of the fields 
  plt.figure(figsize=(15,5),dpi=72)
  plt.subplot(1,6,1); plt.imshow(ds["pwat"].data, cmap="bone");    plt.axis('off'); plt.title("precipitable water")
  plt.subplot(1,6,2); plt.imshow(wind,            cmap="jet");     plt.axis('off'); plt.title("wind speed")
  plt.subplot(1,6,3); plt.imshow(ds["mslp"].data, cmap="plasma");  plt.axis('off'); plt.title("mean sea level pressure")
  plt.subplot(1,6,4); plt.imshow(ds["rh"].data,   cmap="viridis"); plt.axis('off'); plt.title("relative humidity")
  plt.subplot(1,6,5); plt.imshow(ds["lat"].data,  cmap="gray");    plt.axis('off'); plt.title("latitude")
  plt.subplot(1,6,6); plt.imshow(ds["lon"].data,  cmap="gray");    plt.axis('off'); plt.title("longitude")
  plt.subplots_adjust(wspace=0.05, hspace=0)
  plt.show()

**Initialize GPU**

In [ ]:
import torch, nvidia_smi

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device = {device}")

if(device.type =="cuda"):
  nvidia_smi.nvmlInit()
  handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
  print("GPU type =",torch.cuda.get_device_name(0))

**Define utility functions to facilitate training** 

In [ ]:
import torch, os, psutil, pathlib, xarray as xr

#_______________________________________________________________________________
# Dataset: custom set of x,y pairs for Tropical Cyclone classification

class TropicalCycloneDataset(torch.utils.data.Dataset):
  """
  This dataset maps GFS fields as input to cyclone categories as output, for classification. 
  The inputs are read from netcdf files using the given "load function" 

  """
  def __init__(self, source, fields=None, categories=None, load_fcn=None, start=0, stop=1):

    # get input and out fields 
    self.inputs  = ["lat","lon","pwat","mslp","u","v","rh"] if (fields==None) else fields         
    self.outputs = ['TD','TS','CAT1','CAT2','CAT3','CAT4','CAT5','DS','ET','SS','NONE'] if (categories==None) else categories
    self.load_fcn = load_fcn

    # append a list of filenames for each output category
    self.files, self.categories = [],[]
    path = pathlib.Path(source)
    for i,c in enumerate(self.outputs):

      files = sorted(list((path/c).glob('**/*.nc')))
      lo,hi = int(start*len(files)), int(stop*len(files))
      files = files[lo:hi]
      self.files.extend(files)
      self.categories.extend([i]*len(files))
      print(f"Category {c} has {len(files)} items")

    print(f"TCDataset: inputs={self.inputs} outputs={self.outputs} num examples = {len(self.files)}\n")
    
  def __len__(self): 
    return len(self.files)

  def __getitem__(self, index):
    x = self.load_fcn(self.files[index], self.inputs)
    y = torch.tensor(self.categories[index])

    return x,y

#_______________________________________________________________________________
# Loading function for reading NetCDF files

def load_netcdf(file, fields, device="cpu"):
  """
  This function loads data from a netcdf file 
  and stores it iin a torch tensor on the cpu or gpu
  """
  ds   = xr.open_dataset(file) 
  x    = torch.zeros(len(fields), 64, 64, device=device)

  for i,field in enumerate(fields):
    x[i,:,:] = torch.tensor(ds[field].data)
    if(field=="lat"): x[i,:,:]/=90.0
    if(field=="lon"): x[i,:,:]/=360.0
  return x

#_______________________________________________________________________________
# Data Cache for speeding up data access

class Cache():
    """
    This class loads data from RAM if it is available, and reads it from disk
    if it is not. This avoids multiple disk access for the same data.
    """
    def __init__(self, load_fcn, device="cpu", memory_cap = 80):
        self.fcn   = load_fcn
        self.cap   = memory_cap
        self.device= device
        self.cache = {}
        
    def load(self, path, fields):
        
        if path in self.cache: return self.cache[path]
        if (psutil.virtual_memory().percent < self.cap):
            self.cache[path] = self.fcn(path, fields, device)
            return self.cache[path]
        else:
            return self.fcn(file, fields, self.device)

#_______________________________________________________________________________
# Helper functions for keeping track of various measurements

class accuracy_metric():
  
  def __init__(self):
    self.total=0
    self.correct=0
    self.data = []

  def add(self,outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    self.total   += labels.size(0)
    self.correct += (predicted == labels).sum().item()
    
  def value(self): 
    val = self.correct/self.total if self.total else 0
    self.data.extend([val])
    return val

class running_mean():

  def __init__(self):
    self.total = 0
    self.count = 0
  
  def add(self,value):
    self.total+=value
    self.count+=1
  
  def value(self):
    return self.total/self.count if self.count else 0
#_______________________________________________________________________________
# Helper functions for debugging

import torch.nn as nn
class Print(nn.Module):
  def forward(self,x):
    print(x.size())
    return x

def count_parameters(model):
  print("num parameters = ",sum(p.numel() for p in model.parameters() if p.requires_grad))

**Activate Tensorboard, to keep track of our training runs**

In [ ]:
%load_ext tensorboard

# delete old tensorboard log files, if necessary
!rm -r runs

In [ ]:
%tensorboard --logdir runs 

**Define our model**

In [ ]:
import torch, torch.nn as nn

def get_model():

  nrow,ncol = 64, 64 
  C = 3    # number channels in each conv2d
  N = 10   # number neurons in fully connected layer
  K = 3    # convolutional kernel size
  P = K//2 # padding needed to keep the same shape
  
  Cin = len(train_data.inputs) # number of input channels 
  Nout= len(train_data.outputs)# number of output values

  model = nn.Sequential(
      nn.Conv2d(Cin, C,  kernel_size=K,  stride=1, padding=P), nn.MaxPool2d(2,2),nn.ReLU(), 
      nn.Conv2d(C  , C,  kernel_size=K,  stride=1, padding=P), nn.ReLU(), nn.MaxPool2d(2,2),
      nn.Conv2d(C  , C,  kernel_size=K,  stride=1, padding=P), nn.ReLU(), nn.MaxPool2d(2,2),
      nn.Conv2d(C  , C,  kernel_size=K,  stride=1, padding=P), nn.ReLU(), nn.MaxPool2d(2,2),
      nn.Conv2d(C  , C,  kernel_size=K,  stride=1, padding=P), nn.ReLU(), nn.MaxPool2d(2,2),
      nn.Flatten(),
      nn.Linear(C*nrow*ncol//(4*4*4*4*4), N), nn.ReLU(),
      nn.Linear(N, Nout)
      )
  return model

**Train a simple CNN to classify storms**

In [ ]:
'''
TODO: Some activities to try

1. Measure training time: CPU           use_cache=False. runtime->type = None
2. Measure training time: CPU + cache   use_cache=True.  runtime->type = None
3. Measure training time: GPU           use_cache=False. runtime->type = GPU
4. Measure training time: GPU + cache   use_cache=True.  runtime->type = GPU
   Note: when you change runtime type, it deletes your files. So you have to re-download.
   Use nepoch = 2

   training time for epoch 0    = ??, ??, ??, ?? seconds
   training time for epoch 1    = ??, ??, ??, ?? seconds
   training time for 100 epochs = ??, ??, ??, ?? minutes (use: epoch0 + 99 * epoch1)

   (from now on, always use GPU + cache)

5. How accurate is our prediction (use nepochs = 30.)
   val_accuracy = ??

6. How important is the quantity of training data?
          ftrain = 0.1   0.2   0.5   0.9
   val_accuracy  = ??    ??    ??    ??

7. How much can we improve the result by including wind and pressure data?
   fields = ["pwat","u","v","mslp"] val_accuracy = ??

8. How well can we classify  10 categories, using water vapor alone? fields=['pwat']
   Recall, a random guess should get you around 10%
   categories = ['TD','TS','CAT1','CAT2','CAT3','CAT4','CAT5','DS','SS','NONE']
   val_accuracy = ??
''';

In [ ]:
# build data cache.
# you will need to re-run this, if you change fields or categories
cache = Cache(load_netcdf, device, memory_cap=80)

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from time import time
import nvidia_smi
t_start = time()

# USER PARAMETERS

ftrain      = 0.90               # fraction of data to use for training
use_cache   = True               # do we want to use the data cache?
nepochs     = 2                  # number of epochs to train for
fields      = ["pwat"]           # input variables
categories  = ['TS','NONE']      # output variables

# DATA

path         = "/content/crop_64" # location of the data
load_fcn     = cache.load if use_cache else load_netcdf

train_data   = TropicalCycloneDataset(path, fields, categories, load_fcn, start=0.0, stop=ftrain)
val_data     = TropicalCycloneDataset(path, fields, categories, load_fcn, start=ftrain, stop=1.0)

train_loader = DataLoader(train_data, batch_size=200,  shuffle=True,  num_workers=0)
val_loader   = DataLoader(val_data  , batch_size=200,  shuffle=False, num_workers=0)

# MODEL

model = get_model().to(device)
count_parameters(model)

# CONFIGURE

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
loss_fcn  = nn.CrossEntropyLoss()
writer    = SummaryWriter()
acc_list  = []

for epoch in range(nepochs) :

  # VALIDATE
  
  with torch.no_grad():

    model.eval()
    val_loss = running_mean()
    val_acc  = accuracy_metric()

    for i, data in enumerate(val_loader, 0):
      if(epoch==0): print(f"epoch 0: validation batch {i+1}/{len(val_loader)}")

      inputs, labels = data[0].to(device), data[1].to(device)
      outputs   = model(inputs)
      loss      = loss_fcn(outputs,labels)
      val_loss.add(loss.item())
      val_acc.add(outputs,labels)
      
  # TRAIN

  model.train()
  train_acc  = accuracy_metric()
  train_loss = running_mean()
  gpu        = running_mean()
  t0         = time()

  for i, data in enumerate(train_loader, 0):
    if(epoch==0): print(f"epoch 0: training batch {i+1}/{len(train_loader)}")

    inputs, labels = data[0].to(device), data[1].to(device)
    outputs = model(inputs)
    optimizer.zero_grad()
    loss = loss_fcn(outputs,labels)
    loss.backward()
    optimizer.step()

    train_loss.add(loss.item())
    train_acc.add(outputs,labels)
    if(device.type =="cuda"): gpu.add(nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu)

  # DISPLAY PROGRESS

  print(f"epoch={epoch:3d} dt={time()-t0:.3f}s gpu ={gpu.value():.0f}% training loss={train_loss.value():.5f} val loss ={val_loss.value():.5f} train_acc={train_acc.value():.3f} val_acc={val_acc.value():.3f} ")
  writer.add_scalar('Loss/val'      , val_loss.value()  , epoch)
  writer.add_scalar('Loss/train'    , train_loss.value(), epoch)
  writer.add_scalar('Accuracy/val'  , val_acc.value()   , epoch)
  writer.add_scalar('Accuracy/train', train_acc.value() , epoch)
  acc_list.extend( [val_acc.value()] )

print()
print(f"total elapsed training time = { (time()-t_start)/60.0:,.1f} minutes")
print(f"maximum validation accuracy = { max(acc_list) }")